In [4]:
try:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
except:
    pass

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tracker import tracker

cap = cv2.VideoCapture("/home/abhinav/Desktop/ENPM673/Project2/data/project_video.mp4")
K = np.array([[  1.15422732e+03,0.00000000e+00,6.71627794e+02],
              [  0.00000000e+00,1.14818221e+03,3.86046312e+02],
              [  0.00000000e+00,0.00000000e+00,1.00000000e+00]])
dist = np.array([ -2.42565104e-01,-4.77893070e-02,  -1.31388084e-03,  -8.79107779e-05,
    2.20573263e-02])
def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height), max(0,int(center-width)):min(int(center+width),img_ref.shape[1])] = 1
    return output
def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
    res = cv2.LUT(image, lookUpTable)
    return res
def preprocessing(frame):
    th = np.median(frame[int(frame.shape[0]/2):,:,:])
#     frame = adjust_gamma(frame,gamma=1.5)
#     undist = cv2.undistort(frame,K,dist)
#     blur = cv2.bilateralFilter(undist,9,75,75)
#     median = cv2.medianBlur(blur,5)
    
    
#     gray = cv2.cvtColor(median, cv2.COLOR_BGR2HLS)
#     s_channel = gray[:,:,2]
#     l_channel = gray[:,:,1]
    
    #Create mask for S Channel
    if th<60:
        s_channel = adjust_gamma(frame,gamma=25.0)
        undist = cv2.undistort(frame,K,dist)
        blur = cv2.bilateralFilter(undist,9,75,75)
        median = cv2.medianBlur(blur,5)


        gray = cv2.cvtColor(median, cv2.COLOR_RGB2LAB)
        s_channel = gray[:,:,2]
        s_channel = cv2.Sobel(s_channel,cv2.CV_64F,1,0)
        s_channel = np.absolute(s_channel)
        s_channel = np.uint8(255*s_channel/np.max(s_channel))
        l_channel = gray[:,:,0]
        s_min = 30
        s_max = 250

        l_min = 170
        l_max = 255
        
        mask1 = np.zeros([s_channel.shape[0],s_channel.shape[1]])
        mask1[(s_channel>=s_min)&(s_channel<=s_max)]=1
        
        mask2 = np.zeros([l_channel.shape[0],l_channel.shape[1]])
        mask2[(l_channel>=l_min)&(l_channel<=l_max)]=1
    
    elif th>130:
        s_channel = adjust_gamma(frame,gamma=1)
        undist = cv2.undistort(frame,K,dist)
        blur = cv2.bilateralFilter(undist,9,75,75)
        median = cv2.medianBlur(blur,5)


        gray = cv2.cvtColor(median, cv2.COLOR_BGR2LAB)
        s_channel = gray[:,:,2]
        s_channel = gray[:,:,2]
        s_channel = cv2.Sobel(s_channel,cv2.CV_64F,1,0)
        s_channel = np.absolute(s_channel)
        s_channel = np.uint8(255*s_channel/np.max(s_channel))
        l_channel = gray[:,:,0]
        s_min = 20
        s_max = 250
        
        l_min = 220
        l_max = 255
        
        mask1 = np.zeros([s_channel.shape[0],s_channel.shape[1]])
        mask1[(s_channel>=s_min)&(s_channel[1]<=s_max)]=1
        
        
        mask2 = np.zeros([l_channel.shape[0],l_channel.shape[1]])
        mask2[(l_channel>=l_min)&(l_channel<=l_max)]=1
    else:
        s_channel = adjust_gamma(frame,gamma=1.5)
        undist = cv2.undistort(frame,K,dist)
        blur = cv2.bilateralFilter(undist,9,75,75)
        median = cv2.medianBlur(blur,5)


        gray = cv2.cvtColor(median, cv2.COLOR_BGR2LAB)
        s_channel = gray[:,:,2]
        s_channel = cv2.Sobel(s_channel,cv2.CV_64F,1,0)
        s_channel = np.absolute(s_channel)
        s_channel = np.uint8(255*s_channel/np.max(s_channel))
        l_channel = gray[:,:,0]
        s_min = 30
        s_max = 250
        
        l_min = 190
        l_max = 255
        
        mask1 = np.zeros([s_channel.shape[0],s_channel.shape[1]])
        mask1[(s_channel>=s_min)&(s_channel[1]<=s_max)]=1
        
        mask2 = np.zeros([l_channel.shape[0],l_channel.shape[1]])
        mask2[(l_channel>=l_min)&(l_channel<=l_max)]=1
    
    combined = np.zeros([frame.shape[0],frame.shape[1]])
    combined[(mask1==1)|(mask2==1)]=1
    mask = np.uint8(255*combined/np.max(combined))
#     mask = cv2.inRange(median,np.array([0,190,80]),np.array([255,255,150]))
    seg = cv2.bitwise_and(frame,median,mask=mask)
    return seg,combined,th


while(True):
    ret,frame = cap.read()

    seg,combined,th = preprocessing(frame)
    #Extract the region of interest and warp to get bird's eye view
    h = frame.shape[0]
    w = frame.shape[1]
#     src = np.float32([[w-(0.5-0.08/2),h*0.62],[w*(0.5+0.08/2),h*0.62],[w*(0.5+0.76/2),h*0.935],[w*(0.5-0.76/2),h*0.935]])
#     dst = np.array([[0,0],[400,0],[400,600],[0,400]])
    src = np.array([[375,480],[905,480],[1811,685],[-531,685]])
#     dst = np.float32([[w*0.25,0],[0.75*w,0],[0.75*w,h],[0.25*w,h]])
#     src = np.array([[580,450],[160,h],[1150,h],[740,450]])
    dst = np.array([[100,0],[w,0],[w,h-100],[100,h-100]])
    H,flag = cv2.findHomography(src,dst)
    out= cv2.warpPerspective(seg,H,(w,h-100))
    out_gray = cv2.cvtColor(out,cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(out_gray,100,255,cv2.THRESH_BINARY)

#     font = cv2.FONT_HERSHEY_SIMPLEX
#     cv2.putText(out,str(th),(10,500), font, 4,(255,0,0),2,cv2.LINE_AA)
# #     cv2.putText(l_channel,str(th),(10,500), font, 4,(255,0,0),2,cv2.LINE_AA)
#     cv2.putText(combined,str(th),(10,500), font, 4,(255,0,0),2,cv2.LINE_AA)
    ###########################################
    
    
#     window_width = 20
#     window_height = 50
#     curve_centers = tracker(Mywindow_width=window_width, Mywindow_height=window_height, Mymargin = 25, My_ym = 10/720, My_xm = 4/384, Mysmooth_factor=15)
#     window_centroids = curve_centers.find_window_centroids(thresh1)
#     # Points used to draw all the left and right windows
#     l_points = np.zeros([thresh1.shape[0],thresh1.shape[1]])
#     r_points = np.zeros([thresh1.shape[0],thresh1.shape[1]])
        
#     # points used to find the right & left lanes
#     rightx = []
#     leftx = []

#     # Go through each level and draw the windows 
#     for level in range(0,len(window_centroids)/2):
#         # Window_mask is a function to draw window areas
#         # Add center value found in frame to the list of lane points per left, right
#         leftx.append(window_centroids[level][0])
#         rightx.append(window_centroids[level][1])

#         l_mask = window_mask(window_width,window_height,thresh1,window_centroids[level][0],level)
#         r_mask = window_mask(window_width,window_height,thresh1,window_centroids[level][1],level)
#         # Add graphic points from window mask here to total pixels found 
#         l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
#         r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

#     # Draw the results
#     template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
#     zero_channel = np.zeros_like(template) # create a zero color channel
#     template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
#     warpage = np.array(cv2.merge((thresh1,thresh1,thresh1)),np.uint8) # making the original road pixels 3 color channels
#     result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the original road image with window results
    result = window_centers(thresh1,out,window_width=50,window_height=20,margin=25,dst=dst)
    
    
    
    
    
    cv2.imshow('s',result)
    cv2.imshow('l',thresh1)
#     cv2.imshow('test',l_channel)
#     cv2.imshow('mask',mask)
    if cv2.waitKey(1)& 0xff==ord('q'):
        cv2.destroyAllWindows()
        break

/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


IndexError: invalid index to scalar variable.

In [3]:
def window_centers(image,out,window_width,window_height,margin,dst):
    left_windows=[]
    right_windows =[]
    w = image.shape[1]
    h = image.shape[0]
    mid = np.int((dst[0][0]+dst[1][0])/2)
    leftHist = np.sum(image[int(h/2):,:mid],axis=0)
    rightHist = np.sum(image[int(h/2):,mid:],axis=0)
    window = np.ones(window_width)

    left = np.argmax(np.convolve(window,leftHist))-int(window_width/2)
    right = np.argmax(np.convolve(window,rightHist))-int(window_width/2) +mid
    left_windows.append(left)
    right_windows.append(right)
    
    windows = int(h/window_height)
    
    #Search for nonzero pixels in each window
    for win in range(0,int(windows/2)):
        level = np.sum(image[(h - (win+1)*window_height):(h - (win)*window_height),:],axis=0)
        conv = np.convolve(window,level)
        
        x_leftMin = int(max((left-margin+window_width/2),0))
        x_leftMax = int(min((left+margin+window_width/2),mid))
        l_center = np.argmax(conv[x_leftMin:x_leftMax])+int(x_leftMin-window_width/2)
        
        x_rightMin = int(max((right-margin+window_width/2),mid))
        x_rightMax = int(min((right+margin+window_width/2),w))
        r_center = np.argmax(conv[x_rightMin:x_rightMax])+int(x_rightMin-window_width/2)
        
        
        y_min = h - (win+1)*window_height
        y_max = h - (win)*window_height
        
        
        cv2.rectangle(out,(int(l_center),y_min),(int(l_center+window_width),y_max),(0,255,255),2)
        cv2.rectangle(out,(int(r_center),y_min),(int(r_center+window_width),y_max),(255,0,255),2)
        
        left_windows.append(l_center)
        right_windows.append(r_center)
    left_inds=[]
    left_inds = np.average(left_windows[-15:],axis=0)
    right_inds=[]
    right_inds=np.average(right_windows[-15:],axis=0)
        
    return out,left_inds,right_inds